In [4]:
import json
import sumolib
from tqdm import tqdm
from mapmatch import sumo_map_matching, tdf_from_edgelist
from utils import create_xml_vehicles
import folium

#### File paths

In [5]:
# road network path
road_network_path = "../sumo_simulation_data/milan_road_network.net.xml"


# tomtom routed paths path
tomtom_routed_paths_path = "../data/dicts_suggestion/dict_results_tomtom_[0_50].json"


# dict mobility demand path
dict_mobility_demand_path = "../data/dict_mobility_demand.json"


# output routed paths path
output_routed_paths_filename = "../sumo_simulation_data/routed_paths/routed_paths_tomtom.rou.xml"

####  Load road networks

In [6]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)
road_network_int = sumolib.net.readNet(road_network_path, withInternal=True)

#### Load the GPS routed paths

In [7]:
with open(tomtom_routed_paths_path, 'r') as f:
    gps_tomtom_routed_paths = json.load(f)

print(str(len(gps_tomtom_routed_paths))+" GPS routed paths.")

50 GPS routed paths.


####  Load mobility demand dictionary

In [8]:
with open(dict_mobility_demand_path) as json_file:
    dict_md = json.load(json_file)
    
print(str(len(dict_md))+" mobility demands.")

50 mobility demands.


In [9]:
list_id_vehicles = [v_id for v_id in dict_md.keys() if "vehicle" in v_id]

In [11]:
#specify the range of vehicles for which to apply the map matching

id_from, id_to =  0, len(list_id_vehicles)

print("from id "+str(id_from)+" to id "+str(id_to))

from id 0 to id 50


#### Apply the map-matching algorithm

In [12]:
dict_map_matched = {}

pbar = tqdm(total=len(list_id_vehicles[id_from: id_to]))

for v in list_id_vehicles[id_from: id_to]:
    
    points = gps_tomtom_routed_paths[v]['points']
    edge_from = dict_md[v]['edges'][0]
    edge_to = dict_md[v]['edges'][1]
    
    best_edges, sspd_best, length, conf = sumo_map_matching(points, road_network, road_network_path, 
                                                            road_network_int, edge_from, edge_to, accept_th=5e-5,
                                                            early_stop=True)

    
    dict_map_matched[v] = {'mapped_edges':best_edges, 'length':length, 'sspd':sspd_best,
                          'best_conf': conf}
    pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:28<00:00,  1.47it/s]

In [14]:
#for k in dict_map_matched:
#    print(dict_map_matched[k]['sspd']);

### If you want to visually inspect the map matching use the following cell

In [15]:
v_id = "vehicle_1"

points_traj = gps_tomtom_routed_paths[v_id]['points']

m = folium.Map(location=[points_traj[0][1], points_traj[0][0]], tiles='cartodbpositron', zoom_start=13)

# TOMTOM path
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in points_traj], weigth=3, color="blue").add_to(m)

# map matched path
tdf_mapmatched = tdf_from_edgelist(dict_map_matched[v_id]['mapped_edges'], road_network)
points_mapmatched = tdf_mapmatched[['lng','lat']].values

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in points_mapmatched], weigth=3, color="orange").add_to(m)

m

### Create a dictionary describing the mapped edges for each vehicle

In [16]:
dict_sumo_tomtom = {}

for ind, v_id in enumerate(list(dict_md.keys())):      
    if "vehicle" in v_id:
        new_id = "tomtom_"+v_id.split("_")[1]
        dep_time = dict_md[v_id]['time']
        edges_v = dict_map_matched[v_id]['mapped_edges']
        
        dict_sumo_tomtom[new_id] = {'edges':str(edges_v).replace(",","").replace("'","")[1:-1], 'time': dep_time}

### Create a SUMO xml file that described the TomTom routed paths

In [17]:
create_xml_vehicles(dict_sumo_tomtom, output_routed_paths_filename);